# Tutorial 3: Loading and inspecting ME-models

In this tutorial we will load and inspect the reconstructed _Bacillus subtilis_ ME-model.

In [1]:
from IPython.display import display, HTML, Math, Markdown
display(HTML("<style>.container { width:95% !important; }</style>"))

from coralme.builder.main import MEBuilder

/home/jt/UCSD/coralme/coralme/builder/main.py:30: UserWarning: This pandas version does not allow for correct warning handling. Pandas >=1.5.1 is suggested.
  warnings.warn("This pandas version does not allow for correct warning handling. Pandas >=1.5.1 is suggested.")
/home/jt/UCSD/coralme/coralme/builder/preprocess_inputs.py:15: UserWarning: This pandas version does not allow for correct warning handling. Pandas >=1.5.1 is suggested.
  warnings.warn("This pandas version does not allow for correct warning handling. Pandas >=1.5.1 is suggested.")


In [3]:
organism = './organism.json'
inputs = './input.json'

## Load

Load the ME-model coming out of the Troubleshooter

In [16]:
me = MEBuilder.load("",'./MEModel-step3-bacillus-TS.pkl')

Read LP format model from file /tmp/tmphaavk1_9.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Read LP format model from file /tmp/tmp6bj81p9q.lp
Reading time = 0.01 seconds
: 990 rows, 2500 columns, 10478 nonzeros
Read LP format model from file /tmp/tmptbgypwah.lp
Reading time = 0.01 seconds
: 1055 rows, 2496 columns, 10342 nonzeros


## Solve

In [ ]:
me.optimize()

## Inspect

### Summary of the ME-model

In [18]:
me

Name,bacillus
Memory address,0x07f3ed049cb50
Number of metabolites,4245
Number of reactions,7218
Number of groups,0
Objective expression,0
Compartments,"ME-model Constraint, Cytoplasm, Extracellular, im, Periplasm"


In [23]:
print("ME-model has {} reactions".format(len(me.reactions)))

ME-model has 7218 reactions


In [24]:
print("ME-model has {} metabolites".format(len(me.metabolites)))

ME-model has 4245 metabolites


In [25]:
print("ME-model has {} genes".format(len(me.all_genes)))

ME-model has 1042 genes


### Predicted fluxes

In [32]:
fluxes = me.solution.to_frame()
fluxes

,fluxes,reduced_costs
biomass_dilution,9.872746e-02,-5.743958e-01
protein_biomass_to_biomass,5.367014e-02,1.133316e-34
mRNA_biomass_to_biomass,1.749132e-04,-6.591110e-35
tRNA_biomass_to_biomass,1.059782e-03,0.000000e+00
rRNA_biomass_to_biomass,7.708442e-03,0.000000e+00
...,...,...
TS_cobalt2_c,-3.943714e-08,0.000000e+00
TS_zn2_c,-6.446850e-08,0.000000e+00
TS_pydx5p_c,-5.329992e-08,0.000000e+00
TS_fe2_c,-1.273220e-09,0.000000e+00


### Biomass production

In [26]:
fluxes[fluxes.index.str.contains("biomass")]

,fluxes,reduced_costs
biomass_dilution,0.098727,-5.743958e-01
protein_biomass_to_biomass,0.053670,1.133316e-34
mRNA_biomass_to_biomass,0.000175,-6.591110e-35
tRNA_biomass_to_biomass,0.001060,0.000000e+00
rRNA_biomass_to_biomass,0.007708,0.000000e+00
ncRNA_biomass_to_biomass,0.000000,0.000000e+00
tmRNA_biomass_to_biomass,0.000000,0.000000e+00
DNA_biomass_to_biomass,0.005852,0.000000e+00
lipid_biomass_to_biomass,0.000000,0.000000e+00
constituent_biomass_to_biomass,0.000000,0.000000e+00


### Transcription rates

In [29]:
fluxes[fluxes.index.str.contains("^transcription_")].head()

,fluxes,reduced_costs
transcription_TU8J2-1223_from_BSU25200-MONOMER,0.000000e+00,-6.004978e-32
transcription_TU8J2-1268_from_BSU25200-MONOMER,2.232020e-10,-8.029396e-32
transcription_TU8J2-805_from_BSU25200-MONOMER,0.000000e+00,-2.295000e+02
transcription_TU8J2-1022_from_BSU25200-MONOMER,0.000000e+00,-4.380787e+02
transcription_TU8J2-458_from_BSU25200-MONOMER,2.471596e-08,-3.732743e-32


### Translation rates

In [30]:
fluxes[fluxes.index.str.contains("^translation_")].head()

,fluxes,reduced_costs
translation_BSU00090,4.314142e-09,4.619552e-32
translation_BSU00110,0.000000e+00,-1.747777e-32
translation_BSU00120,0.000000e+00,7.897237e-33
translation_BSU00130,3.497031e-08,-5.603378e-33
translation_BSU00140,0.000000e+00,1.813539e-33


### Sinks to be curated through manual curation

The Troubleshooter found 8 gaps that need to be curated to allow for growth. Most of these are cofactors that will require some synthesis pathways (e.g. 4fe4s), but others will only need some transporters (e.g. metal ions)

In [20]:
fluxes[fluxes.index.str.contains("^TS_")]

,fluxes,reduced_costs
TS_cbl1_c,-4.087457e-09,0.0
TS_4fe4s_c,-2.445478e-07,0.0
TS_thmpp_c,-7.019815e-09,0.0
TS_cobalt2_c,-3.943714e-08,0.0
TS_zn2_c,-6.446850e-08,0.0
TS_pydx5p_c,-5.329992e-08,0.0
TS_fe2_c,-1.273220e-09,0.0
TS_mn2_c,-2.740831e-06,0.0


### Mapped gene functions

In [48]:
from collections import defaultdict
import pandas
d = defaultdict(int)
for g in me.all_genes:
    for f in g.functions:
        d[f] += 1
pandas.DataFrame.from_dict({"count":d})

,count
Metabolic:S_Amino_acids_and_related_molecules,155
Metabolic:S_Carbohydrates_and_related_molecules,155
Metabolic:S_Cell_wall,40
Metabolic:S_Coenzymes_and_prosthetic_groups,77
Metabolic:S_Lipids,55
Metabolic:S_Membrane_bioenergetics,47
Metabolic:S_Nucleotides_and_nucleic_acids,69
Metabolic:S_Other_function,15
Metabolic:S_Phosphate_and_sulfur,7
Metabolic:S_Transport,232


#### Inspecting functions of a gene

In [43]:
g = me.all_genes.get_by_id("RNA_BSU37160")
g.functions

{'Transcription'}

### Complexes formed by a gene

In [43]:
g = me.all_genes.get_by_id("RNA_BSU37160")
g.functions

{'Transcription'}